<div style="border: 2px solid #ccc; border-radius: 12px; padding: 20px; max-width: 950px; margin: auto; background-color: #1e1e1e; color: #f0f0f0; font-family: Arial, sans-serif; line-height: 1.6;">

  <div style="text-align: center; margin-bottom: 20px;">
    <img src="..\images\SlideHunter_Logo.png" 
         alt="Coffee Production Boxplot by Subdivision"
         style="width: 80%; max-width: 80%; height: auto; border-radius: 8px; box-shadow: 0 0 10px rgba(0,0,0,0.4);">
  </div>

  <blockquote style="margin: 0; padding: 10px 20px; border-left: 4px solid #4faaff;">
    <p><strong>
      SlideHunter App
    </strong></p>
    <p>
     User Interface (UI) : 
      <a href="..\images\SlideHunter_Logo.png" target="_blank" style="color: #4faaff;">
        Find exactly where a topic was covered in course materials. Fast answers with precise slide/page citations.
      </a>
    </p>
  </blockquote>

</div>

## Check if GPU is alalable for faster embedding

In [1]:
import torch, platform

print("cuda_available:", torch.cuda.is_available())
print("torch.cuda:", torch.version.cuda)          # None if CUDA isn't available
print("device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU only")


cuda_available: False
torch.cuda: None
device: CPU only


# 01 — Setup & Ingest Notebook

## Retrieval-Augmented-Generation (RAG)

Retrieval-Augmented Generation is a *technique* to improve an LLM's responses by:
* Retrieving relevant documents from a knowledge store (such FAISS or a vector DB).
* Augmenting the model's prompt with those documents.
* Generating an answer using the model with this extra context.

With RAG, the model is handed the right documents at generation time. That is, the model does not respond to a user's queries until it refers to a specified set of documents.

The FAISS store acts as our long-term memory for domain knowledge. The RAG pipeline serves as the reasoning loop: it retrieves the most relevant content, assembles the top results into a compact context, and passes that context to the LLM to produce a grounded, citeable response.
- Think of FAISS as the library and RAG as the librarian: FAISS shelves all the course knowledge; RAG finds the right books, marks the key pages, and hands them to the model to explain.

This notebook parses Canvas Learning Management System (LMS) (cours-level), embeds chunks, and builds a FAISS Store 


## Install (optional)

In [2]:
# Warning surpresser
import os

# Tell Hugging Face to skip TensorFlow/Flax so they never import TensorFlow (TF).
os.environ["TRANSFORMERS_NO_TF"] = "1"
os.environ["TRANSFORMERS_NO_FLAX"] = "1"

# Quiet TF logs if something still pulls it in.
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"  # 1=INFO, 2=WARNING, 3=ERROR

## Imports and Credentials

In [3]:
# OPTIONAL If env is missing packages, or in a fresh/new environment
#%pip install -q -r requirements.txt

In [4]:
#import json, os
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
import numpy as np, faiss, re, json, os
from canvasapi import Canvas
import torch

# Load environment variables from .env file
from dotenv import dotenv_values

In [5]:
# Load environment variables from .env file
config = dotenv_values() # load .env file

# Injest and process data from Canvas Sections
# Set up Canva API client
CANVAS_BASE_URL = config.get("CANVAS_BASE_URL")
CANVAS_TOKEN=config.get("CANVAS_TOKEN")
OPENAI_API_KEY = config.get("OPENAI_API_KEY")

# Initialize Canvas API client 
canvas = Canvas(CANVAS_BASE_URL, CANVAS_TOKEN)

## Injest and process data from Canvas Sections

In [6]:
# Getting the list of courses
my_courses = canvas.get_courses()

In [7]:
# Pulling courses from Canvas
my_courses = canvas.get_courses()
course_list = []

for course in my_courses:
    print(course.name)
    course_list.append(course)


Foundations '25 Data Science
Foundations Course
IF '25 Data Science Cohort A
IF '25 NY Career Readiness and Success


In [8]:
# Pulling modules from courses on Canvas 
modules = course.get_modules()

for module in modules:
    print(f"  Module_id: {module.id}")
    print(f"  Module: {module.name}")
    module_items = module.get_module_items()
    for item in module_items:
        print(f" - Item: {item.title} ({item.type})")

  Module_id: 1118
  Module: Fellow Resources
 - Item: Fellow Success Resources (Page)
  Module_id: 1239
  Module: Phase 2 (6/9-8/29)
 - Item: Homework: Option 1 - Weekly Job Applications & Progress Report (Due August 30) (Assignment)
 - Item: P2W1 (6/12) NO CAREER CLASS - TECHNICAL CLASS (SubHeader)
 - Item: P2W2 (6/16) Bloomberg Ideathon (SubHeader)
 - Item: Homework (SubHeader)
 - Item: Homework: Watch Hackathon Video (Assignment)
 - Item: Homework: Upwardly Global Learning Paths: Tech Market/Resume/Cover Letter (Assignment)
 - Item: Homework: Draft Resume (Assignment)
 - Item: P2W2 NO CLASS MEETING 6/19 Juneteenth TKH Closed (SubHeader)
 - Item: P2W3 (6/26) Bloomberg Hackathon (SubHeader)
 - Item: Homework (SubHeader)
 - Item: Homework: Hackathon Activity Log + Judges' Feedback (Assignment)
 - Item: P2W4 (7/3) Resume + Digital Footprint (SubHeader)
 - Item: In Class Activity (SubHeader)
 - Item: In-Class Activity: Updated Resume (Assignment)
 - Item: Homework (SubHeader)
 - Item: Ho

## Embedding Tokenized Canvas modules (Texts/items). Then Turning Those Embddings into a facts list + FAISS index that we can query.

1. Build facts (+ metadata) from Canvas
  - This pulls Pages' text (HTML → plain text), and adds light facts for  
    External URLs / Files / SubHeaders → we may have to extend this later.

## single FAISS store:
- Simple/Demo MVP, which tags every fact with a domain and use a tiny auto-router
  - Two-way short route descriptions (technical.index and career.index)
    - Pulls multiple Canvas courses
    - Builds one facts/metas list with domain in metadata
    - And creates one FAISS index
- This method routes queries to technical / career / all--automatically and filters hits accordingly.



In [9]:
# Multi-course to ONE FAISS store + simple router using career and technical courses

# 0) CONFIG: map course names to domain buckets

DOMAINS = {
    "technical": [
        "Foundations '25 Data Science",
        "Foundations Course",
        "IF '25 Data Science Cohort A",
    ],
    "career": [
        "IF '25 NY Career Readiness and Success",
    ],
}
# Short route descriptions--We can add more if needed (used for auto routing purpos)
ROUTE_DESC = {
    "technical": "Technical class content: Python, SQL, statistics, machine learning, slides, labs, code, algorithms, data science, lecture notes.",
    "career":    "Career readiness content: resumes, cover letters, job search, interviews, career prep, LinkedIn, networking, internship resources.",
}

In [ ]:
# 1) Utility: HTML → text, light chunking
def strip_html(html: str) -> str:
    """Removes HTML tags and returns a single string.

    This function parses an HTML string, extracts the text, and joins
    it into a single string. It handles whitespace by
    replacing multiple spaces with a single space and stripping leading/
    trailing whitespace.

    Args:
        html (str): The HTML content to be processed.

    Returns:
        str: The plain-text content without HTML tags.
    """
    if not html: return ""
    txt = " ".join(BeautifulSoup(html, "html.parser").stripped_strings)
    return re.sub(r"\s+", " ", txt).strip()

def chunk_text(text, max_chars=600):
    """Splits a long string into a list of smaller text chunks.

    This function chunks a text string based on a maximum character
    limit. It attempts to split the text at natural breaks, such as
    newlines or the end of a sentence (followed by a period and a space),
    to create more readable chunks.

    Args:
        text (str): The text to be chunked.
        max_chars (int, optional): The maximum number of characters
            for each chunk. Defaults to 600.

    Returns:
        list[str]: A list of text chunks, where each chunk is
            no longer than `max_chars`.
    """
    if not text: return []
    parts = re.split(r"(\n|\.\s+)", text)
    buf, chunks = "", []
    for p in parts:
        buf += p
        if len(buf) >= max_chars:
            chunks.append(buf.strip()); buf = ""
    if buf.strip(): chunks.append(buf.strip())
    return [c for c in chunks if c]

In [ ]:
# 2) Select courses by name (use your Canvas client `canvas`)

def course_domain(course_name: str):
    """
    Finds the index of the most recent entry in a DataFrame.
    It returns the integer index
    of this last row.

    Args:
        df (pd.DataFrame): The pandas DataFrame to analyze.

    Returns:
        int: The integer index of the most recent entry.
    """
    for dom, names in DOMAINS.items():
        if any(course_name.startswith(n) for n in names):
            return dom
    return "other"

wanted_prefixes = sum(DOMAINS.values(), [])
all_courses = [c for c in canvas.get_courses(enrollment_state="active")
               if any(c.name.startswith(p) for p in wanted_prefixes)]

print("Selected courses:", [c.name for c in all_courses])

Selected courses: ["Foundations '25 Data Science", 'Foundations Course', "IF '25 Data Science Cohort A", "IF '25 NY Career Readiness and Success"]


In [12]:
# 3) Build facts + metas from ALL selected courses
facts, metas = [], []
for crs in all_courses:
    dom = course_domain(crs.name)
    print(f"Course: {crs.name} (domain={dom})")
    for module in crs.get_modules():
        for item in module.get_module_items():
            t = (item.type or "").strip()
            if t == "Page":
                page = crs.get_page(item.page_url)
                text = strip_html(getattr(page, "body", ""))
                for chunk in chunk_text(text, max_chars=600):
                    facts.append(f"[{dom}] {crs.name} > {module.name} > {item.title}: {chunk}")
                    metas.append({
                        "domain": dom,
                        "course_id": crs.id, "course_name": crs.name,
                        "module_id": module.id, "module_name": module.name,
                        "item_title": item.title, "type": "Page",
                        "url": getattr(page, "html_url", None)
                    })
            elif t in ("ExternalUrl", "ExternalTool"):
                facts.append(f"[{dom}] {crs.name} > {module.name} > {item.title}: external link {getattr(item, 'external_url', '')}")
                metas.append({
                    "domain": dom, "course_id": crs.id, "course_name": crs.name,
                    "module_id": module.id, "module_name": module.name,
                    "item_title": item.title, "type": t,
                    "url": getattr(item, "external_url", None)
                })
            elif t == "File":
                facts.append(f"[{dom}] {crs.name} > {module.name} > {item.title} (file)")
                metas.append({
                    "domain": dom, "course_id": crs.id, "course_name": crs.name,
                    "module_id": module.id, "module_name": module.name,
                    "item_title": item.title, "type": "File", "file_id": item.content_id
                })
            elif t == "SubHeader":
                continue
            else:
                facts.append(f"[{dom}] {crs.name} > {module.name} > {item.title} ({t})")
                metas.append({
                    "domain": dom, "course_id": crs.id, "course_name": crs.name,
                    "module_id": module.id, "module_name": module.name,
                    "item_title": item.title, "type": t
                })

print(f"Built {len(facts)} facts")

Course: Foundations '25 Data Science (domain=technical)
Course: Foundations Course (domain=technical)
Course: IF '25 Data Science Cohort A (domain=technical)
Course: IF '25 NY Career Readiness and Success (domain=career)
Built 324 facts


In [13]:
# 4) Embed — use GPU if available, else CPU
import torch
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device=DEVICE)
print("model device:", model.device)

# (optional) quick warm-up on GPU
if DEVICE == "cuda":
    _ = model.encode(["warm up"], show_progress_bar=False)

# pick a sensible batch size per device
BATCH = 192 if DEVICE == "cuda" else 64

emb = model.encode(
    facts,
    batch_size=BATCH,
    normalize_embeddings=True,   # cosine-ready
    convert_to_numpy=True,       # returns NumPy on CPU for FAISS
    show_progress_bar=True
).astype("float32")

d = emb.shape[1]
index = faiss.IndexFlatIP(d)               # cosine (vectors normalized)
index.add(emb)
print("FAISS ntotal:", index.ntotal)


model device: cpu


Batches:   0%|          | 0/6 [00:00<?, ?it/s]

FAISS ntotal: 324


In [ ]:
# 5) Router: choose technical / career / all based on similarity to route descriptions
route_emb = {k: model.encode([v], normalize_embeddings=True).astype("float32") for k,v in ROUTE_DESC.items()}

def choose_scope(query, margin=0.05):
    """
    Selects the most relevant content category for a given user query.

    This function compares a user's question or search query to predefined
    categories (like "technical" or "career") to find the best match. It
    uses the "sentence transformer" to understand
    the meaning of the words.

    The function returns the best-matching category. However, if two
    categories are very close in relevance (within a small "margin"), it
    chooses "all" to be safe, meaning the query could apply to both.

    Args:
        query (str): The user's question or search query.
        margin (float, optional): A small number that decides how close
                                  two categories can be before the function
                                  chooses "all". Defaults to 0.05.

    Returns:
        Tuple[str, Dict[str, float]]: A tuple containing:
            - The name of the best-matching category (e.g., "technical" or "all").
            - A dictionary showing the similarity score for each category.
    """
    q = model.encode([query], normalize_embeddings=True).astype("float32")
    sims = {k: float((q @ route_emb[k].T)[0,0]) for k in ROUTE_DESC}
    best = max(sims, key=sims.get)
    # if not clearly better, use 'all'
    ordered = sorted(sims.items(), key=lambda x: x[1], reverse=True)
    if ordered[0][1] - ordered[1][1] < margin:
        return "all", sims
    return best, sims

In [ ]:
# 6) Search with optional scope filter (auto by default)
def search(query, k=5, scope="auto"):
    """Determines the most relevant content scope for a given query.

    This function uses semantic similarity to compare a user's query against
    predefined scope descriptions. It selects the best-matching scope (
    "technical" or "career"). If the top two scores are within the specified
    margin of each other, it returns "all" to indicate ambiguity.

    Args:
        query (str): The user's input query string.
        margin (float, optional): The similarity threshold for determining
                                  ambiguity between scopes. Defaults to 0.05.

    Returns:
        Tuple[str, Dict[str, float]]: A tuple containing:
            - The selected scope string ("technical", "career", or "all").
            - A dictionary of similarity scores for each scope.
    """
    if scope == "auto":
        scope, sims = choose_scope(query)
    q = model.encode([query], normalize_embeddings=True).astype("float32")
    # pull more then filter by domain
    D, I = index.search(q, k*8)
    hits = []
    for score, idx in zip(D[0], I[0]):
        if idx == -1: continue
        m = metas[idx]
        if scope != "all" and m["domain"] != scope:
            continue
        hits.append({"score": float(score), "fact": facts[idx], "meta": m})
        if len(hits) >= k: break
    # if not enough in-scope, backfill with any
    if len(hits) < k:
        for score, idx in zip(D[0], I[0]):
            if idx == -1: continue
            if any(h["meta"] is metas[idx] for h in hits): continue
            hits.append({"score": float(score), "fact": facts[idx], "meta": metas[idx]})
            if len(hits) >= k: break
    return scope, hits

In [16]:
# 7) Try it out with some pre-test test-prompts
tests = [
    "Where did we define precision vs. recall?",
    "tips for a resume and cover letter?",
    "What lecture slides did we learn about control flow?",
  ]
for q in tests:
    scope, hits = search(q, k=4, scope="auto")
    print(f"\nQ: {q}   [scope={scope}]")
    if not hits: print("  (no hits)"); continue
    for h in hits:
        m = h["meta"]
        cite = f"{m['course_name']} > {m['module_name']} > {m['item_title']} ({m['type']})"
        if m.get("url"): cite += f"  [{m['url']}]"
        print(f"  {h['score']:.3f} :: {cite}")



Q: Where did we define precision vs. recall?   [scope=technical]
  0.382 :: IF '25 Data Science Cohort A > P2W3 (6/23-6/27) Classification Algorithms > 💻 W3D2 (6/24) Logistic Regression Accuracy Metrics (Page)  [https://tkh.instructure.com/courses/172/pages/w3d2-6-slash-24-logistic-regression-accuracy-metrics]
  0.306 :: Foundations '25 Data Science > Week 5:  Statistics(Feb. 24th- Feb. 27th) > What is Data Science? (Page)  [https://tkh.instructure.com/courses/165/pages/what-is-data-science]
  0.276 :: IF '25 Data Science Cohort A > P2W11 (8/18-8/22) Agents & End of Phase Project > 💻 W11D1 (8/18) Applied LLM Review & AI Agents (Page)  [https://tkh.instructure.com/courses/172/pages/w11d1-8-slash-18-applied-llm-review-and-ai-agents]
  0.263 :: IF '25 Data Science Cohort A > P2W9 (8/4-8/8) NLP Foundations & Transformers > 📚 P2W9 Overview & Lesson Plan (Page)  [https://tkh.instructure.com/courses/172/pages/p2w9-overview-and-lesson-plan]

Q: tips for a resume and cover letter?   [scope=car

In [ ]:
# --- Persist FAISS + metadata to the repo root (SLIDEHUNTER/) ---
from pathlib import Path
import os, json, faiss

# 0) Resolve project base: prefer env var; else step out of notebooks/
ENV_BASE = os.getenv("SLIDEHUNTER_BASE") or os.getenv("SLIDEHUNT_BASE")
if ENV_BASE:
    BASE = Path(ENV_BASE).resolve()
else:
    here = Path.cwd().resolve()
    BASE = here.parent if here.name.lower() == "notebooks" else here  # run from repo root if you're inside notebooks/

# 1) Paths under the repo
STORE_DIR  = BASE / "data" / "faiss"
INDEX_PATH = STORE_DIR / "canvas.index"
FACTS_PATH = STORE_DIR / "facts.json"
STORE_DIR.mkdir(parents=True, exist_ok=True)

# 2) Save / Load helpers
def save_store(index, facts, metas, index_path=INDEX_PATH, facts_path=FACTS_PATH):
    """Saves the FAISS index, facts, and metadata.

    This function serializes and saves the vector search index and its
    corresponding data. The FAISS index is saved in its native binary format,
    while the facts and their metadata are stored together in a JSON file.

    Args:
        index (faiss.Index): The FAISS index to be saved.
        facts (List[str]): A list of text strings corresponding to the facts.
        metas (List[Dict]): A list of dictionaries containing metadata for
                            each fact.
        index_path (Path, optional): The file path to save the FAISS index.
                                     Defaults to INDEX_PATH.
        facts_path (Path, optional): The file path to save the facts and
                                     metadata. Defaults to FACTS_PATH.
    """
    faiss.write_index(index, str(index_path))
    with open(facts_path, "w", encoding="utf-8") as f:
        json.dump({"facts": facts, "metas": metas}, f, ensure_ascii=False)
    print("saved:", index_path)
    print("saved:", facts_path)

def load_store(index_path=INDEX_PATH, facts_path=FACTS_PATH):
    """Loads your saved search data from your computer.

    This function opens the files that were saved by `save_store` and brings
    your FAISS index, facts, and metadata back into the program.
    This is much faster than having to create all the data again.

    Args:
        index_path (Path, optional): The location of the saved index file.
        facts_path (Path, optional): The location of the saved facts file.

    Returns:
        Tuple[faiss.Index, List[str], List[Dict]]: The loaded index, facts,
                                                     and metadata.
    """
    idx = faiss.read_index(str(index_path))
    data = json.load(open(facts_path, "r", encoding="utf-8"))
    print("loaded:", index_path, "and", facts_path)
    return idx, data["facts"], data["metas"]

# 3) Save right after you build `index`, `facts`, `metas`
save_store(index, facts, metas)


saved: C:\Users\MINA-G\SlideHunter\data\faiss\canvas.index
saved: C:\Users\MINA-G\SlideHunter\data\faiss\facts.json
